In [ ]:

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import *


In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np

In [ ]:

import env
reload(env)
import agent
reload(agent)
import sensors
reload(sensors)
import brain
reload(brain)

In [ ]:

b = env.env(600,600, show=False)

In [ ]:
b.load_from_yaml('data.yml')

In [ ]:
a = agent.agent(x=15,y=15, ray_dist=50, ray_count=4, ray_width=70, show=True)

In [ ]:
b.add_agent(a)

In [12]:
# brain go or don't go
# turn which dir and how much
import tensorflow as tf

X = tf.placeholder("float", [None, 11])
Y = tf.placeholder("float", [None, 2])

rng = np.random
rand_array = np.random.rand(1, 11)
# Set model weights
W = tf.Variable( rng.randn(), name="weight")
b = tf.Variable(rng.randn(), name="bias")

#pred = tf.add(tf.matmul(X, W), b)

sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())

In [3]:
print rand_array

[[ 0.19933511  0.50265161  0.21277225  0.29581334  0.19241594  0.95503379
   0.1962772   0.73090485  0.17022613  0.23765714  0.8360665 ]]


In [13]:
a = sess.run(W, feed_dict={X: rand_array})

In [15]:
a

0.51585436

In [ ]:

import time

start = time.time()
num_tries = 100
for i in range(num_tries):
    observations = a.observe()
    for t in range(100):
        action = a.brain.decide(observations)
        observations, reward, done, info = a.step(action)
   
        if done:
            break

    a.reset()
end  = time.time()
print (end-start)

In [ ]:
#for i_episode in range(20):
#    observation = env.reset()
#    for t in range(100):
#        env.render()
#        print(observation)
#        action = env.action_space.sample()
#        observation, reward, done, info = env.step(action)
#        if done:
#            print("Episode finished after {} timesteps".format(t+1))
#            break

In [ ]:
import pygame
import sys


pygame.init()

In [ ]:
screen = pygame.display.set_mode((640,480))

In [ ]:

pygame.display.update()
screen.fill(black)

In [ ]:
red = (255,0,0)
green = (0,255,0)
blue = (0,0,255)
darkBlue = (0,0,128)
white = (255,255,255)
black = (0,0,0)
pink = (255,200,200)

In [ ]:

pygame.draw.circle(screen, red,  (50,100), 25, 1)


In [ ]:
pygame.quit(); sys.exit();

In [ ]:
while (True):

   # check for quit events
   for event in pygame.event.get():
        if event.type == pygame.QUIT:
             pygame.quit(); sys.exit();

   # erase the screen
   screen.fill(white)

   # draw the updated picture

   updatePoints(points)  # changes the location of the points
   pygame.draw.lines(screen,black,false,points,1)  # redraw the points

   # update the screen
   pygame.display.update()

In [ ]:
int((34.4,56))

In [ ]:
def fun(a, z=None):

    b = z or 'a'
    print b

In [ ]:
fun(2, z=(1,2,4))

In [ ]:
[1,2,3,4][1::2]

In [ ]:
#%matplotlib inline
import tensorflow as tf
import numpy as np
import random
import gym
import math
import matplotlib.pyplot as plt


def softmax(x):
    e_x = np.exp(x - np.max(x))
    out = e_x / e_x.sum()
    return out


def policy_gradient():
    with tf.variable_scope("policy", reuse=True):
        params = tf.get_variable("policy_parameters",[4,2])
        state = tf.placeholder("float",[None,4])
        actions = tf.placeholder("float",[None,2])
        advantages = tf.placeholder("float",[None,1])
        linear = tf.matmul(state,params)
        probabilities = tf.nn.softmax(linear)
        good_probabilities = tf.reduce_sum(tf.mul(probabilities, actions),reduction_indices=[1])
        eligibility = tf.log(good_probabilities) * advantages
        loss = -tf.reduce_sum(eligibility)
        optimizer = tf.train.AdamOptimizer(0.01).minimize(loss)
        return probabilities, state, actions, advantages, optimizer

def value_gradient():
    with tf.variable_scope("value",reuse=True):
        state = tf.placeholder("float",[None,4])
        newvals = tf.placeholder("float",[None,1])
        w1 = tf.get_variable("w1",[4,10])
        b1 = tf.get_variable("b1",[10])
        h1 = tf.nn.relu(tf.matmul(state,w1) + b1)
        w2 = tf.get_variable("w2",[10,1])
        b2 = tf.get_variable("b2",[1])
        calculated = tf.matmul(h1,w2) + b2
        diffs = calculated - newvals
        loss = tf.nn.l2_loss(diffs)
        optimizer = tf.train.AdamOptimizer(0.1).minimize(loss)
        return calculated, state, newvals, optimizer, loss

def run_episode(env, policy_grad, value_grad, sess):
    pl_calculated, pl_state, pl_actions, pl_advantages, pl_optimizer = policy_grad
    vl_calculated, vl_state, vl_newvals, vl_optimizer, vl_loss = value_grad
    observation = env.reset()
    totalreward = 0
    states = []
    actions = []
    advantages = []
    transitions = []
    update_vals = []


    for _ in xrange(200):
        # calculate policy
        obs_vector = np.expand_dims(observation, axis=0)
        probs = sess.run(pl_calculated,feed_dict={pl_state: obs_vector})
        action = 0 if random.uniform(0,1) < probs[0][0] else 1
        # record the transition
        states.append(observation)
        actionblank = np.zeros(2)
        actionblank[action] = 1
        actions.append(actionblank)
        # take the action in the environment
        old_observation = observation
        observation, reward, done, info = env.step(action)
        transitions.append((old_observation, action, reward))
        totalreward += reward

        if done:
            break
    for index, trans in enumerate(transitions):
        obs, action, reward = trans

        # calculate discounted monte-carlo return
        future_reward = 0
        future_transitions = len(transitions) - index
        decrease = 1
        for index2 in xrange(future_transitions):
            future_reward += transitions[(index2) + index][2] * decrease
            decrease = decrease * 0.97
        obs_vector = np.expand_dims(obs, axis=0)
        currentval = sess.run(vl_calculated,feed_dict={vl_state: obs_vector})[0][0]

        # advantage: how much better was this action than normal
        advantages.append(future_reward - currentval)

        # update the value function towards new return
        update_vals.append(future_reward)

    # update value function
    update_vals_vector = np.expand_dims(update_vals, axis=1)
    sess.run(vl_optimizer, feed_dict={vl_state: states, vl_newvals: update_vals_vector})
    # real_vl_loss = sess.run(vl_loss, feed_dict={vl_state: states, vl_newvals: update_vals_vector})

    advantages_vector = np.expand_dims(advantages, axis=1)
    sess.run(pl_optimizer, feed_dict={pl_state: states, pl_advantages: advantages_vector, pl_actions: actions})

    return totalreward


env = gym.make('CartPole-v0')

policy_grad = policy_gradient()
value_grad = value_gradient()
sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())
for i in xrange(200):
    reward = run_episode(env, policy_grad, value_grad, sess)
    if reward == 200:
        print "reward 200"
        print i
        break
t = 0
for _ in xrange(100):
    reward = run_episode(env, policy_grad, value_grad, sess)
    t += reward
print t / 100
